In [1]:
from datasets import load_dataset
from torchvision import datasets
import os

In [2]:
datasets.CIFAR10(root='~/data/10', download=True)

100%|██████████| 170M/170M [00:06<00:00, 25.1MB/s] 


Extracting /home/jovyan/data/10/cifar-10-python.tar.gz to /home/jovyan/data/10


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /home/jovyan/data/10
    Split: Train

In [3]:
datasets.CIFAR100(root='~/data/100', download=True)

100%|██████████| 169M/169M [00:06<00:00, 25.0MB/s] 


Extracting /home/jovyan/data/100/cifar-100-python.tar.gz to /home/jovyan/data/100


Dataset CIFAR100
    Number of datapoints: 50000
    Root location: /home/jovyan/data/100
    Split: Train

In [4]:
sst2_train = load_dataset('stanfordnlp/sst2', split='train')
sst2_eval = load_dataset('stanfordnlp/sst2', split='validation')
sst2_test_blank = load_dataset('stanfordnlp/sst2', split='test')

README.md:   0%|          | 0.00/5.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [5]:
data = sst2_train.train_test_split(test_size=0.2, seed=42, stratify_by_column='label')

In [6]:
sst2_train = data['train']
sst2_test = data['test']

In [7]:
sst2_train.save_to_disk("~/data/sst2/train")
sst2_eval.save_to_disk("~/data/sst2/eval")
sst2_test.save_to_disk("~/data/sst2/test")

sst2_test_blank.save_to_disk("~/data/sst2/test-blank")

Saving the dataset (0/1 shards):   0%|          | 0/53879 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/872 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13470 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1821 [00:00<?, ? examples/s]

In [8]:
dbpedia_train = load_dataset("fancyzhx/dbpedia_14", split='train')
dbpedia_test = load_dataset("fancyzhx/dbpedia_14", split='test') 
data = dbpedia_train.train_test_split(test_size=0.2, seed=42, stratify_by_column='label')

README.md:   0%|          | 0.00/7.64k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/106M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/13.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/70000 [00:00<?, ? examples/s]

In [9]:
dbpedia_train = data['train']
dbpedia_eval = data['test']

dbpedia_train = dbpedia_train.rename_column("content", "sentence")
dbpedia_train = dbpedia_train.remove_columns(["title"])

dbpedia_eval = dbpedia_eval.rename_column("content", "sentence")
dbpedia_eval = dbpedia_eval.remove_columns(["title"])

dbpedia_test = dbpedia_test.rename_column("content", "sentence")
dbpedia_test = dbpedia_test.remove_columns(["title"])

In [10]:
dbpedia_train.save_to_disk("~/data/dbpedia/train")
dbpedia_eval.save_to_disk("~/data/dbpedia/eval")
dbpedia_test.save_to_disk("~/data/dbpedia/test")

Saving the dataset (0/1 shards):   0%|          | 0/448000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/112000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/70000 [00:00<?, ? examples/s]

In [11]:
trec_train_coarse = load_dataset("CogComp/trec", split='train') 
trec_test_coarse = load_dataset("CogComp/trec", split='test') 

data = trec_train_coarse.train_test_split(test_size=0.2, seed=42, stratify_by_column='coarse_label')

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

trec.py:   0%|          | 0.00/5.09k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5452 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [12]:
trec_train_coarse = data['train']
trec_eval_coarse = data['test']

trec_train_coarse = trec_train_coarse.rename_column("text", "sentence")
trec_train_coarse = trec_train_coarse.rename_column("coarse_label", "label")
trec_train_coarse = trec_train_coarse.remove_columns(["fine_label"])

trec_eval_coarse = trec_eval_coarse.rename_column("text", "sentence")
trec_eval_coarse = trec_eval_coarse.rename_column("coarse_label", "label")
trec_eval_coarse = trec_eval_coarse.remove_columns(["fine_label"])

trec_test_coarse = trec_test_coarse.rename_column("text", "sentence")
trec_test_coarse = trec_test_coarse.rename_column("coarse_label", "label")
trec_test_coarse = trec_test_coarse.remove_columns(["fine_label"])

In [13]:
trec_train_coarse.save_to_disk("~/data/trec/train_coarse")
trec_eval_coarse.save_to_disk("~/data/trec/eval_coarse")
trec_test_coarse.save_to_disk("~/data/trec/test_coarse")

Saving the dataset (0/1 shards):   0%|          | 0/4361 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1091 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

In [14]:
trec_train_coarse = load_dataset("CogComp/trec", split='train') 
trec_test_coarse = load_dataset("CogComp/trec", split='test') 

data = trec_train_coarse.train_test_split(test_size=0.2, seed=42, stratify_by_column='fine_label')

In [15]:
trec_train_coarse = data['train']
trec_eval_coarse = data['test']

trec_train_coarse = trec_train_coarse.rename_column("text", "sentence")
trec_train_coarse = trec_train_coarse.rename_column("fine_label", "label")
trec_train_coarse = trec_train_coarse.remove_columns(["coarse_label"])

trec_eval_coarse = trec_eval_coarse.rename_column("text", "sentence")
trec_eval_coarse = trec_eval_coarse.rename_column("fine_label", "label")
trec_eval_coarse = trec_eval_coarse.remove_columns(["coarse_label"])

trec_test_coarse = trec_test_coarse.rename_column("text", "sentence")
trec_test_coarse = trec_test_coarse.rename_column("fine_label", "label")
trec_test_coarse = trec_test_coarse.remove_columns(["coarse_label"])

In [16]:
trec_train_coarse.save_to_disk("~/data/trec/train_fine")
trec_eval_coarse.save_to_disk("~/data/trec/eval_fine")
trec_test_coarse.save_to_disk("~/data/trec/test_fine")

Saving the dataset (0/1 shards):   0%|          | 0/4361 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1091 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
try:
    os.mkdir(f"{os.path.expanduser('~')}/models")
    os.mkdir(f"{os.path.expanduser('~')}/models/cifar10/")
    os.mkdir(f"{os.path.expanduser('~')}/models/cifar100/")
    os.mkdir(f"{os.path.expanduser('~')}/models/sst2/")
    os.mkdir(f"{os.path.expanduser('~')}/models/dbpedia/")
    os.mkdir(f"{os.path.expanduser('~')}/models/trec/")
except:
    print("Folder structure has been already created!")

In [ ]:
try:
    os.mkdir(f"{os.path.expanduser('~')}/data/10-logits/")
    os.mkdir(f"{os.path.expanduser('~')}/data/10-logits/cifar-10-batches-py/")
    os.mkdir(f"{os.path.expanduser('~')}/data/100-logits/")
    os.mkdir(f"{os.path.expanduser('~')}/data/100-logits/cifar-100-python/")
except:
    print("Folder structure has been already created!")